In [27]:
# Imports
import pandas as pd
import numpy as np

from datetime import datetime

# Genre import
import ast

In [3]:
# Importing Billboard Hot 100 Dataset (contains y-target) and Song Features (contains X-features)
df = pd.read_csv('Hot Stuff.csv')
features_df = pd.read_excel('Hot 100 Audio Features.xlsx')

# Importing tracks dataset to balance dataset before model building
df2 = pd.read_csv('data/tracks.csv')


print(df.shape,features_df.shape, df2.shape)

(327895, 10) (29503, 22) (586672, 20)


In [4]:
hot100 = df.copy()
hot100_features = features_df.copy()
tracks = df2.copy()

In [5]:
# url, weekly position, instance, previous week position are all irrelevant to my goals.
hot100.drop(['url', 'Week Position', 'Instance', 'Previous Week Position'], axis=1, inplace=True)

# Url not needed. Album could be used for future projects, not for time crunch tasks. Popularity -> Data Leakage
hot100_features.drop(['spotify_track_preview_url', 'spotify_track_album', 'spotify_track_popularity'], axis=1, inplace=True)

# popularity might lead to data leakage
tracks.drop(['popularity'], axis=1, inplace=True)

In [6]:
# Rename Columns and extract relevant strings
tracks['artists'] = tracks['artists'].apply(lambda x: x[2:-2])
tracks['id_artists'] = tracks['id_artists'].apply(lambda x: x[2:-2])

tracks.rename(columns={'name':'Song', 'artists':'Performer',
                       'id':'spotify_track_id'}, inplace=True)

In [7]:
perfect100 = hot100.groupby('SongID', group_keys=True).apply(lambda x: x.loc[x['Weeks on Chart'].idxmax()])

In [8]:
perfect100.shape

(29389, 6)

In [10]:
perfect100.reset_index(inplace =True, drop = True)

In [12]:
# Converts WeekID entries (type str) to (type datetime)
perfect100['WeekID'] = pd.to_datetime(perfect100['WeekID'])

In [13]:
hot100_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29503 entries, 0 to 29502
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SongID                     29503 non-null  object 
 1   Performer                  29503 non-null  object 
 2   Song                       29503 non-null  object 
 3   spotify_genre              27903 non-null  object 
 4   spotify_track_id           24397 non-null  object 
 5   spotify_track_duration_ms  24397 non-null  float64
 6   spotify_track_explicit     24397 non-null  float64
 7   danceability               24334 non-null  float64
 8   energy                     24334 non-null  float64
 9   key                        24334 non-null  float64
 10  loudness                   24334 non-null  float64
 11  mode                       24334 non-null  float64
 12  speechiness                24334 non-null  float64
 13  acousticness               24334 non-null  flo

In [14]:
hot100_features.dropna(inplace =True)

In [15]:
# I'm good with the amount lost. Features-wise : 24334 - 24186 = 0.00666. Less than 1%.
hot100_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24186 entries, 2 to 29502
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SongID                     24186 non-null  object 
 1   Performer                  24186 non-null  object 
 2   Song                       24186 non-null  object 
 3   spotify_genre              24186 non-null  object 
 4   spotify_track_id           24186 non-null  object 
 5   spotify_track_duration_ms  24186 non-null  float64
 6   spotify_track_explicit     24186 non-null  float64
 7   danceability               24186 non-null  float64
 8   energy                     24186 non-null  float64
 9   key                        24186 non-null  float64
 10  loudness                   24186 non-null  float64
 11  mode                       24186 non-null  float64
 12  speechiness                24186 non-null  float64
 13  acousticness               24186 non-null  flo

In [16]:
perfect100.shape, hot100_features.shape

((29389, 6), (24186, 19))

In [17]:
combo = pd.merge(perfect100, hot100_features, how='right', on ='SongID')
combo.shape

(24186, 24)

In [18]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24186 entries, 0 to 24185
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   WeekID                     24185 non-null  datetime64[ns]
 1   Song_x                     24185 non-null  object        
 2   Performer_x                24185 non-null  object        
 3   SongID                     24186 non-null  object        
 4   Peak Position              24185 non-null  float64       
 5   Weeks on Chart             24185 non-null  float64       
 6   Performer_y                24186 non-null  object        
 7   Song_y                     24186 non-null  object        
 8   spotify_genre              24186 non-null  object        
 9   spotify_track_id           24186 non-null  object        
 10  spotify_track_duration_ms  24186 non-null  float64       
 11  spotify_track_explicit     24186 non-null  float64       
 12  danc

In [20]:
combo[combo['Song_x'].isna()]

,WeekID,Song_x,Performer_x,SongID,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature


In [21]:
combo.dropna(inplace =True)

In [22]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24185 entries, 0 to 24185
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   WeekID                     24185 non-null  datetime64[ns]
 1   Song_x                     24185 non-null  object        
 2   Performer_x                24185 non-null  object        
 3   SongID                     24185 non-null  object        
 4   Peak Position              24185 non-null  float64       
 5   Weeks on Chart             24185 non-null  float64       
 6   Performer_y                24185 non-null  object        
 7   Song_y                     24185 non-null  object        
 8   spotify_genre              24185 non-null  object        
 9   spotify_track_id           24185 non-null  object        
 10  spotify_track_duration_ms  24185 non-null  float64       
 11  spotify_track_explicit     24185 non-null  float64       
 12  danc

In [1]:
# Fix Duplicate Columns resultant from Merge. 
combo.drop(['Song_y','Performer_y'], axis = 1, inplace=True)
combo.rename(columns={'Song_x':'Song', 'Performer_x':'Performer'}, 
             inplace=True)

In [26]:
# Set new column for future binary classification
combo['top100'] = 1

In [ ]:
type(combo['spotify_genre'][10])

In [28]:
# Fix Genre column to actual lists
combo['spotify_genre'] = combo['spotify_genre'].apply(lambda x: ast.literal_eval(x))

In [29]:
type(combo['spotify_genre'][10])

list

## Balancing Dataset

In [30]:
# Combo, contains all the features of the unique Hot 100 songs
# Tracks contains all the features of songs that may or may not be in the Hot 100.
combo.shape, tracks.shape

((24185, 23), (586672, 19))

In [31]:
tracks[tracks['spotify_track_id'] == "6or1bKJiZ06IlK0vFvY75k"]

,spotify_track_id,Song,duration_ms,explicit,Performer,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
86073,6or1bKJiZ06IlK0vFvY75k,Rap God,363521,1,Eminem,7dGJo4pcD2V6oG8kP0tJRR,2013-11-05,0.708,0.843,7,-2.66,1,0.314,0.397,0.0,0.799,0.625,148.14,4


In [32]:
combo[combo['spotify_track_id'] == "6or1bKJiZ06IlK0vFvY75k"]

,WeekID,Song,Performer,SongID,Peak Position,Weeks on Chart,spotify_genre,spotify_track_id,spotify_track_duration_ms,spotify_track_explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,top100
16054,2014-03-15,Rap God,Eminem,Rap GodEminem,7.0,20.0,"[detroit hip hop, g funk, hip hop, rap]",6or1bKJiZ06IlK0vFvY75k,363521.0,1.0,...,-2.66,1.0,0.314,0.397,0.0,0.799,0.625,148.14,4.0,1


In [33]:
# Merge. Correctly this time to avoid duplicate columns. 
model_ready = pd.merge(combo, tracks, how='outer', 
                   left_on=['Song', 'Performer', 'spotify_track_id', 'danceability', 'energy', 'key', 
                            'loudness', 'mode',
                            'speechiness','acousticness','instrumentalness', 'liveness', 'valence',
                            'tempo', 'time_signature'], 
                   right_on=['Song', 'Performer', 'spotify_track_id', 'danceability', 'energy', 'key', 
                             'loudness', 'mode',
                            'speechiness','acousticness','instrumentalness', 'liveness', 'valence',
                            'tempo', 'time_signature'])
model_ready.shape

(605766, 27)

In [34]:
model_ready[model_ready['spotify_track_id'] == "6or1bKJiZ06IlK0vFvY75k"]

,WeekID,Song,Performer,SongID,Peak Position,Weeks on Chart,spotify_genre,spotify_track_id,spotify_track_duration_ms,spotify_track_explicit,...,instrumentalness,liveness,valence,tempo,time_signature,top100,duration_ms,explicit,id_artists,release_date
16053,2014-03-15,Rap God,Eminem,Rap GodEminem,7.0,20.0,"[detroit hip hop, g funk, hip hop, rap]",6or1bKJiZ06IlK0vFvY75k,363521.0,1.0,...,0.0,0.799,0.625,148.14,4.0,1.0,363521.0,1.0,7dGJo4pcD2V6oG8kP0tJRR,2013-11-05


In [35]:
model_ready.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 605766 entries, 0 to 605765
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   WeekID                     24185 non-null   datetime64[ns]
 1   Song                       605695 non-null  object        
 2   Performer                  605766 non-null  object        
 3   SongID                     24185 non-null   object        
 4   Peak Position              24185 non-null   float64       
 5   Weeks on Chart             24185 non-null   float64       
 6   spotify_genre              24185 non-null   object        
 7   spotify_track_id           605766 non-null  object        
 8   spotify_track_duration_ms  24185 non-null   float64       
 9   spotify_track_explicit     24185 non-null   float64       
 10  danceability               605766 non-null  float64       
 11  energy                     605766 non-null  float64 

In [36]:
# Songs that didn't sync up are songs not in the Hot 100. 
# On merge, values for them on columns that didn't exist become nan
model_ready['top100'].unique()

array([ 1., nan])

In [37]:
# Replace nan values with 0. (mainly for column 'top100')
model_ready['top100'] = model_ready['top100'].fillna(0)

In [38]:
# Check if nan's are converted/ nan is in any column we care about.
model_ready[model_ready.isna().any(axis=1)]

,WeekID,Song,Performer,SongID,Peak Position,Weeks on Chart,spotify_genre,spotify_track_id,spotify_track_duration_ms,spotify_track_explicit,...,instrumentalness,liveness,valence,tempo,time_signature,top100,duration_ms,explicit,id_artists,release_date
0,1965-05-15,......And Roses And Roses,Andy Williams,......And Roses And RosesAndy Williams,36.0,7.0,"[adult standards, brill building pop, easy lis...",3tvqPPpXyIgKrm4PR9HCf0,166106.0,0.0,...,0.000267,0.1120,0.1500,83.969,4.0,1.0,NaN,NaN,NaN,NaN
5,1976-05-29,'til I Can Make It On My Own,Tammy Wynette,'til I Can Make It On My OwnTammy Wynette,84.0,5.0,"[country, country dawn, nashville sound]",0aJHZYjwbfTmeyUWF7zGxI,182080.0,0.0,...,0.000035,0.1080,0.1460,141.148,4.0,1.0,NaN,NaN,NaN,NaN
7,2010-05-15,'Til Summer Comes Around,Keith Urban,'Til Summer Comes AroundKeith Urban,58.0,16.0,"[australian country, contemporary country, cou...",1CKmI1IQjVEVB3F7VmJmM3,331466.0,0.0,...,0.000136,0.7700,0.3080,127.907,4.0,1.0,NaN,NaN,NaN,NaN
8,1995-12-02,'Til You Do Me Right,After 7,'Til You Do Me RightAfter 7,31.0,22.0,"[funk, neo soul, new jack swing, quiet storm, ...",3kGMziz884MLV1oCwrarmN,295000.0,0.0,...,0.000000,0.0763,0.4330,76.744,4.0,1.0,NaN,NaN,NaN,NaN
9,1962-01-27,'Til,The Angels,'TilThe Angels,14.0,15.0,"[blues rock, garage rock, modern blues rock, n...",6cMBpazQUSw6xYiLNYvQzf,148440.0,0.0,...,0.000180,0.0939,0.3070,173.819,3.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605761,NaT,云与海,阿YueYue,NaN,NaN,NaN,NaN,5rgu12WBIHQtvej2MdHSH0,NaN,NaN,...,0.000000,0.0648,0.2110,131.896,4.0,0.0,258267.0,0.0,1QLBXKM5GCpyQQSVMNZqrZ,2020-09-26
605762,NaT,blind,ROLE MODEL,NaN,NaN,NaN,NaN,0NuWgxEp51CutD2pJoF4OM,NaN,NaN,...,0.000297,0.0924,0.6860,150.091,4.0,0.0,153293.0,0.0,1dy5WNgIKQU6ezkpZs4y8z,2020-10-21
605763,NaT,What They'll Say About Us,FINNEAS,NaN,NaN,NaN,NaN,27Y1N4Q4U3EfDU5Ubw8ws2,NaN,NaN,...,0.000150,0.0874,0.0663,145.095,4.0,0.0,187601.0,0.0,37M5pPGs6V1fchFJSgCguX,2020-09-02
605764,NaT,A Day At A Time,"Gentle Bones', 'Clara Benin",NaN,NaN,NaN,NaN,45XJsGpFTyzbzeWK8VzR8S,NaN,NaN,...,0.000003,0.3050,0.4380,90.029,4.0,0.0,142003.0,0.0,"4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2gkqjn",2021-03-05


In [39]:
# Check count for songs in the Top 100
model_ready[model_ready['top100'] == 1.0].count()

WeekID                       24185
Song                         24185
Performer                    24185
SongID                       24185
Peak Position                24185
Weeks on Chart               24185
spotify_genre                24185
spotify_track_id             24185
spotify_track_duration_ms    24185
spotify_track_explicit       24185
danceability                 24185
energy                       24185
key                          24185
loudness                     24185
mode                         24185
speechiness                  24185
acousticness                 24185
instrumentalness             24185
liveness                     24185
valence                      24185
tempo                        24185
time_signature               24185
top100                       24185
duration_ms                   5132
explicit                      5132
id_artists                    5132
release_date                  5132
dtype: int64

In [40]:
# Check count for songs not in the Top 100
model_ready[model_ready['top100'] == 0.0].count()

WeekID                            0
Song                         581510
Performer                    581581
SongID                            0
Peak Position                     0
Weeks on Chart                    0
spotify_genre                     0
spotify_track_id             581581
spotify_track_duration_ms         0
spotify_track_explicit            0
danceability                 581581
energy                       581581
key                          581581
loudness                     581581
mode                         581581
speechiness                  581581
acousticness                 581581
instrumentalness             581581
liveness                     581581
valence                      581581
tempo                        581581
time_signature               581581
top100                       581581
duration_ms                  581581
explicit                     581581
id_artists                   581581
release_date                 581581
dtype: int64

In [42]:
model_ready.sort_values('top100', ascending =False, inplace=True)

In [43]:
# Pay attention to Top100 when switching between 24184-24186
model_ready.loc[24185]  # log_reg.loc[24186] 

WeekID                                          NaT
Song                                          Carve
Performer                                       Uli
SongID                                          NaN
Peak Position                                   NaN
Weeks on Chart                                  NaN
spotify_genre                                   NaN
spotify_track_id             35iwgR4jXetI318WEWsa1Q
spotify_track_duration_ms                       NaN
spotify_track_explicit                          NaN
danceability                                  0.645
energy                                        0.445
key                                             0.0
loudness                                    -13.338
mode                                            1.0
speechiness                                   0.451
acousticness                                  0.674
instrumentalness                              0.744
liveness                                      0.151
valence     

In [51]:
# Merge for a perfect Dataset to predict Top100.
model_ready_df = pd.concat([model_ready[:24185],
                          model_ready[24185:].sample(24190)])

In [52]:
# Check New dataset
model_ready_df[model_ready_df['top100'] == 1.0].count()

WeekID                       24185
Song                         24185
Performer                    24185
SongID                       24185
Peak Position                24185
Weeks on Chart               24185
spotify_genre                24185
spotify_track_id             24185
spotify_track_duration_ms    24185
spotify_track_explicit       24185
danceability                 24185
energy                       24185
key                          24185
loudness                     24185
mode                         24185
speechiness                  24185
acousticness                 24185
instrumentalness             24185
liveness                     24185
valence                      24185
tempo                        24185
time_signature               24185
top100                       24185
duration_ms                   5132
explicit                      5132
id_artists                    5132
release_date                  5132
dtype: int64

In [53]:
model_ready_df[model_ready_df['top100'] == 0.0].count()

WeekID                           0
Song                         24190
Performer                    24190
SongID                           0
Peak Position                    0
Weeks on Chart                   0
spotify_genre                    0
spotify_track_id             24190
spotify_track_duration_ms        0
spotify_track_explicit           0
danceability                 24190
energy                       24190
key                          24190
loudness                     24190
mode                         24190
speechiness                  24190
acousticness                 24190
instrumentalness             24190
liveness                     24190
valence                      24190
tempo                        24190
time_signature               24190
top100                       24190
duration_ms                  24190
explicit                     24190
id_artists                   24190
release_date                 24190
dtype: int64

In [ ]:
model_ready_df.to_csv("cleaned_and_balanced_top100")

## End